<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week2/Keras_Tutorial_v2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras tutorial - Emotion Detection in Images of Faces #

2주차 첫 번째 과제에 오신것을 환영합니다! 이 과제에서 여러분은,

1. Python으로 작성되고 TensorFlow 및 CNTK를 비롯한 여러 하위 수준 프레임 워크에서 실행할 수있는 고수준 신경망 API (프로그래밍 프레임 워크) 인 Keras를 사용하는 방법을 배웁니다.
2. 몇 시간 안에 딥 러닝 알고리즘을 구축하는 방법을 배워보세요.

**왜 Keras를 사용하나요?**

- Keras는 딥 러닝 엔지니어가 다양한 모델을 매우 빠르게 구축하고 실험 할 수 있도록 개발되었습니다.
- TensorFlow가 Python보다 높은 수준의 프레임 워크 인 것처럼 Keras는 훨씬 더 높은 수준의 프레임 워크이며 추가 추상화를 제공합니다.
- 가능한 한 최소한의 지연으로 아이디어에서 결과로 이동할 수있는 것이 좋은 모델을 찾는 데 중요합니다.
- 그러나 Keras는 하위 수준 프레임 워크보다 더 제한적이므로 Keras가 아닌 TensorFlow에서 여전히 구현할 매우 복잡한 모델이 있습니다.
- 즉, Keras는 많은 일반적인 모델에서 잘 작동합니다.



## Load packages ##

- 이 과제에서, 여러분은 아래 설명되어 있는"감정 인식" 모델을 개발할 것입니다.
- 필요한 패키지들을 불러와봅시다.

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline


**참고** : 보다시피 Keras에서 많은 함수를 가져 왔습니다. 코드에서 직접 호출하여 사용할 수 있습니다. 예 :`X = Input (...)`또는`X = ZeroPadding2D (...)`.

즉, TensorFlow와 달리 그래프를 만든 다음 별도의`sess.run()` 호출을 수행하여 해당 변수를 평가할 필요가 없습니다.

## 1 - Emotion Tracking ##


- 인근 지역 사회 보건소에서 지역 주민들의 정신 건강 모니터링을 돕고 있습니다.
- 연구의 일환으로 자원 봉사자들에게 하루 종일 자신의 감정을 기록하도록 요청하고 있습니다.
- 참가자가 자신의 감정을 더 쉽게 추적 할 수 있도록 지원자가 얼굴 표정을 촬영할 사진을 기반으로 감정을 분류하는 앱을 만들어야합니다.
- 그 증거로서 먼저 누군가의 감정이 "행복"또는 "행복하지 않음"으로 분류되는지 감지하도록 모델을 훈련시킵니다.

이 모델을 구축하고 훈련하기 위해 인근 지역에서 자원 봉사자들의 사진을 수집했습니다. 데이터 세트에 라벨이 지정됩니다.

<img src="arts/face_images.png" style="width:550px;height:250px;">

다음 코드를 실행하여 데이터 세트를 정규화하고 데이터의 shape 에 대해 알아 봅니다.


In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

**"Face" 데이터셋의 세부사항**:
- (64, 64, 3)의 shape를 가진 이미지로 이루어져 있습니다.
- 훈련 세트 : 이미지 600장
- 테스트 세트 : 이미지 150장

## 2 - Building a model in Keras ##

케라스는 빠른 프로토타입 제작에 매우 훌륭하게 사용될 수 있습니다. 모델을 만들고 유의미한 결과를 내기까지 잠깐의 시간이면 충분합니다.

아래는 케라스를 사용한 모델 예시입니다.

```python
def model(input_shape):
    """
    input_shape: The height, width and channels as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]
    """
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model
```

**네이밍 규칙**:

- Keras는 지금까지 우리가 봤던 numpy와 Tensorflow와는 다른 네이밍 컨벤션을 가지고 있습니다. 
- 아래처럼 각 단계와 레이어별로 다른 변수 이름을 선언했던것과 달리,
  ```python
  X = ...
  Z1 = ...
  A1 = ...
  ```
- Keras는 각 단계별로 같은 변수를 재사용하고, 덮어씁니다.
 ```python
 X = ...
 X = ...
 X = ...
 ```
- 네이밍 규칙의 예외는 `X_input`입니다. 이 값은 나중에 필요하므로 별도로 보관하겠습니다.

#### 함수로서의 객체


- 각 코드에 두 쌍의 괄호가 있음을 확인하십시오. 예를 들면 :
```python
X = ZeroPadding2D((3, 3))(X_input)
```
- 첫 번째는 ZeroPadding2D 객체를 생성하는 생성자 호출부입니다.
- 파이썬에서, 객체는 함수라는 다른 이름으로도 불릴 수 있습니다. 
'python object as function'를 검색하면 블로그 게시물 [Python Pandemonium](https://medium.com/python-pandemonium/function-as-objects-in-python-d5215e6d1b0d)을 읽을 수 있습니다. "함수로서의 객체"섹션을 참조하십시오.
- 위 한 줄짜리 코드는 풀어쓰면 아래와 같습니다.
```python
ZP = ZeroPadding2D((3, 3))   # ZP is an object that can be called as a function
X = ZP(X_input)
```

**연습 문제** : `HappyModel()`을 구현해보세요.

* 이 과제는 다른 대부분의 과제보다 더 개방적입니다.
* 제시된 아키텍처를 사용하여 모델을 구현하는 것으로 시작하고,이를 초기 모델로 사용하여 이 과제의 나머지 부분을 실행합니다. 
* 나중에 돌아와서 다른 모델 아키텍처를 사용해보십시오.
* 예를 들어, 위 모델에서 영감을 얻은 다음 원하는대로 네트워크 아키텍처와 하이퍼 파라미터를 변경할 수 있습니다.
* `AveragePooling2D()`, `GlobalMaxPooling2D()`, `Dropout()` 과 같은 다른 함수도 사용할 수 있습니다.

In [ ]:
# GRADED FUNCTION: HappyModel

def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset
        (height, width, channels) as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]

    Returns:
    model -- a Model() instance in Keras
    """
    
    ### START CODE HERE ###
    # Feel free to use the suggested outline in the text above to get started, and run through the whole
    # exercise (including the later portions of this notebook) once. The come back also try out other
    # network architectures as well. 
    
    
    ### END CODE HERE ###
    
    return model

이제 모델을 설명하는 함수를 빌드했습니다. 이 모델을 학습하고 테스트하기 위해 Keras에는 다음 네 단계가 있습니다.

1. 위의 함수를 호출하여 모델을 만듭니다.

2. `model.compile(optimizer = "...", loss = "...", metrics = [ "accuracy"])` 을 호출하여 모델을 컴파일합니다.

3. `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)` 를 호출하여 훈련 데이터에서 모델을 훈련시킵니다.

4. `model.evaluate(x = ..., y = ...)` 를 호출하여 테스트 데이터에서 모델을 테스트합니다.

`model.compile()`, `model.fit()`, `model.evaluate()`및 해당 인수에 대한 자세한 내용은 공식 [Keras 문서](https://keras.io/models/model) 를 참조하세요.

#### Step 1: create the model ####

**힌트**
`input_shape` 파라미터는 (높이, 너비, 채널 수)의 shape를 가진 튜플입니다. 배치 사이즈를 제외한 값이므로, `X_train.shape[1:] 를 `input_shape`로 설정해야 합니다.

In [ ]:
### START CODE HERE ### (1 line)
happyModel = None
### END CODE HERE ###

#### Step 2: compile the model

**힌트**

사용할 수 있는 최적화 함수에는` 'adam'`,`'sgd'`  등이 있습니다. [optimizers](https://keras.io/optimizers/) 에 대한 문서를 참조하세요.
"Happiness detection"는 이진 분류 문제입니다. 사용할 수있는 손실 함수는` 'binary_cross_entropy'`입니다. `categorical_cross_entropy`는 형식이 지정된 데이터 세트에서 작동하지 않습니다. 데이터가 두 개의 배열 (각 카테고리에 하나씩)이 아니라 0 또는 1의 배열이기 때문입니다. 자세한 사항은 [손실 문서를 참조하세요](https://keras.io/losses/)

In [ ]:
### START CODE HERE ### (1 line)
None
### END CODE HERE ###

#### Step 3: train the model

**힌트**
`X_train`, `Y_train` 변수를 사용하세요. epoch와 batch_size 값으로는 integer 변수가 들어가야 합니다.

**참고**
`fit()` 를 다시 실행하면 모델은 다시 초기화하는 대신 이미 학습 한 매개 변수로 계속 학습합니다.

In [ ]:
### START CODE HERE ### (1 line)
None
### END CODE HERE ###

#### Step 4: evaluate model ####

**힌트**:
`X_test`와 `Y_text` 변수를 써서 모델의 성능을 측정해보세요.

In [ ]:
### START CODE HERE ### (1 line)
preds = None
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

#### Expected performance ####

만약 `happyModel()` 함수가 잘 동작한다면, 정확도는 무작위 추측의 정확도 50%보다 더 높은 확률의 정확도를 가져야 합니다,

비교할만한 수치 하나만 제공하자면, 우리가 직접 구현한 모델은 **미니 배치 사이즈 16, adam 최적화 함수로 40번의 epoch 이후 95%(훈련 세트에선 99%)의 정확도를 보였습니다**

#### Tips for improving your model ####

만약 여러분의 모델이 좋은 성능(80% 정확도 이상)을 보이지 못하고 있다면, 아래 몇 가지 팁을 참조하세요.

- CONV -> BATCHNORM -> RELU로 이어지는 블록을 사용해보세요. 코드는 아래와 같습니다.
```python
X = Conv2D(32, (3, 3), strides=(1,1), name='conv0')(X)
X = BatchNormalization(axis=3, name='bn0')(X)
X = Activation('relu')(X)
```
이 작업을 높이와 너비가 작아지고, 채널 수가 비교적 커질 때(약 32개) 까지 반복하십시오
그 이후 3차원 volume을 평면화하여 fully-connected layer로 만드세요.

- 위의 블록 이후 MAXPOOL을 사용하세요, 높이와 너비의 차원을 낮추는데 도움이 됩니다.

- 최적화 함수를 다른 것으로 변경해보세요. 'Adam' 최적화 함수가 좋은 성능을 가진다는것을 배운 적이 있을 것입니다.

- 메모리 문제가 발생했다면, batch_size를 줄여보세요(ex, 12).

- 훈련 정확도가 더이상 향상되지 않을 때 까지 더 많이 반복(epoch) 하세요.


**참고** : 모델에서 하이퍼파라미터 튜닝을 수행하면 테스트 세트가 실제로 개발 세트가되고 모델이 테스트 (dev) 세트에 과적 합 될 수 있습니다. 일반적으로 별도의 개발 및 테스트 세트가 필요합니다. 개발 세트는 매개 변수 조정에 사용되며 테스트 세트는 프로덕션에서 모델의 성능을 추정하는 데 한 번 사용됩니다.


## 3 - Conclusion ##

축하합니다! "happiness detection" 모델을 개발 완료했습니다

#### Key Points to remeber ####

- Keras는 빠른 프로토타이핑을 위해서 추천할 수 있는 도구입니다. 이 프레임워크는 서로 다른 모델 아키텍쳐를 빠르게 구현해볼 수 있도록 합니다.
- 아래 네 가지 스텝을 기억하세요.
  1. Create
  2. Compile
  3. Fit/Train
  4. Evalutate/Test

## 7 - 자체 이미지를 통한 테스트 (선택 학습 / 채점 안됨) ##

과제를 모두 마치신 것을 축하드립니다! 이제 직접 자신이 가지고 있는 이미지를 가지고 모델의 output을 확인해봅시다. 아래의 방법을 따라해보세요.

1. 이 notebook 의 상단 표시줄에서 "File"을 클릭 후, "open"을 클릭하여 Cousera Hub로 이동합니다.
2. "images" 폴더에 있는 Jupyter Notebook의 디렉토리에 이미지를 추가합니다.
3. 아래 코드에서 이미지 이름을 변경합니다.
4. 코드를 실행하고, 알고리즘이 올바른 결과를 내는지 확인하세요!

In [ ]:
## START CODE HERE ##
my_image = "my_image.jpg" # change this to the name of your image file 
my_label_y = [1] # the true class of your image (1 -> cat, 0 -> non-cat)
## END CODE HERE ##

fname = "images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
my_image = my_image/255.
my_predicted_image = predict(my_image, my_label_y, parameters)

plt.imshow(image)
print ("y = " + str(np.squeeze(my_predicted_image)) + ", your L-layer model predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")